---
date: 2024-04-29
title: Experiments in the Irrationality of Sqrt 2 with SMT
---

A classical theorem is that $\sqrt{2}$ is irrational. https://en.wikipedia.org/wiki/Square_root_of_2#Proofs_of_irrationality This means it cannot be written as $\frac{p}{q}$ for integers $p$ and $q$.

This and the existence of an infinite number of primes are some simple prominent theorems and reasonable test cases for a system intended to support proofs.

There is an interesting article [The Seventeen Provers of the World](https://www.cs.ru.nl/~freek/comparison/comparison.pdf) compiled by Freek Wiedijk which does the irrationality of two in many different systems.

I've been interested in trying to build infrastructure for a highly automated interactive proof system using off the shelf solvers and this is an test case.

First off, just asking the question in a naive-ish way chokes the solvers. It is not inconceivable that they could have baked this in.

In [1]:
from z3 import *
# from cvc5.pythonic import * # cvc5 is a pretty good dropin. It also fails here.
n,m = Ints("n m")
prove(Not(Exists([n,m], And(m != 0, n*n == 2 * m * m))))

failed to prove


Z3Exception: model is not available

Ok, well we may have to help it along.

I took a look in Baby Rudin to see what their proof looks like. This led me down a rabbithole that was seemingly leading no where. 

I started proving lemmas about even and odd numbers. There were some basic facts that seemed difficult to push through.


In [ ]:
even = Function("even", IntSort(), BoolSort())
even_def = ForAll(x, even(x) == Exists([y], x == 2*y))
odd = Function("odd", IntSort(), BoolSort())
odd_def = ForAll(x, odd(x) == Exists([y], x == 2*y+1))

even_or_odd = ForAll(x, even(x) != odd(x))
# Z3 can't prove this for some reason. I suppose this might be a complex diophantine equation or something.
# CVC5 seemingly can prove this.
prove(Implies(And(even_def, odd_def), even_or_odd))

div2_even = ForAll([x], even(x) == (x == 2 * (x/2)))
prove(Implies(even_def, div2_even))

div2_odd = ForAll([x], odd(x) == (x == 2 * (x/2) + 1))
prove(Implies(odd_def, div2_odd))

So instead I took a look at the Otter proof in the above paper. Otter is an automated theorem and the predecessor of [Prover9](https://www.cs.unm.edu/~mccune/prover9/manual/2009-11A/). It offers some features with more operational flavor that the modern contingent of solvers (vampire and eprover) don't.

This encoding of the problem seems pretty clever. `d(x,y)` is the y is divisible by x. `m` is multiplication. I think implicitly it is working over positive integers to avoid `!= 0` constraints.

Note that in CNF, `~ foo | bar` should often be read as `foo => bar` which is classically equivalent. In that case, this reads as some equations and horn clauses / conditional equations.

In [2]:
%%file /tmp/sqrt2.in
% From the Wiedijk review. Wos, Beeson, McCune
set(auto).
%set(ur_res).
%assign(max_distinct_vars, 1).
formulas(sos).
    x = x.  %? Why did they write this?
    m(1,x) = x. % identity
    m(x,1) = x.
    m(x,m(y,z)) = m(m(x,y),z). % associativity
    m(x,y) = m(y,x). % commutativity
    m(x,y) != m(x,z) | y = z. % cancellation... ??? What about x = 0? Implicit positive integers only?
    -d(x,y) | m(x,f(x,y)) = y. % this and next line define divides
    m(x,z) != y | d(x,y).
    -d(2,m(x,y)) | d(2,x) | d(2,y). % 2 is prime
    m(a,a) = m(2,m(b,b)). % a/b = √2
    -d(x,a) | -d(x,b) | x = 1. % a/b is in lowest terms
    2 != 1. % I almost forgot this!
end_of_list.

Overwriting /tmp/sqrt2.in


In [4]:
! prover9 -f /tmp/sqrt2.in

============================== Prover9 ===============================
Prover9 (64) version 2017-11A (CIIRC), November 2017.
Process 370431 was started by philip on philip-ThinkPad-Z13-Gen-2,
Mon Apr 29 20:47:34 2024
The command was "prover9 -f /tmp/sqrt2.in".
============================== end of head ===========================

============================== INPUT =================================

% Reading from file /tmp/sqrt2.in

set(auto).
    % set(auto) -> set(auto_inference).
    % set(auto) -> set(auto_setup).
    % set(auto_setup) -> set(predicate_elim).
    % set(auto_setup) -> assign(eq_defs, unfold).
    % set(auto) -> set(auto_limits).
    % set(auto_limits) -> assign(max_weight, "100.000").
    % set(auto_limits) -> assign(sos_limit, 20000).
    % set(auto) -> set(auto_denials).
    % set(auto) -> set(auto_process).

formulas(sos).
x = x.
m(1,x) = x.
m(x,1) = x.
m(x,m(y,z)) = m(m(x,y),z).
m(x,y) = m(y,x).
m(x,y) != m(x,z) | y = z.
-d(x,y) | m(x,f(x,y)) = y.
m(x,z) != y

This problem is solved instantly. I thought the same problem given to vampire or eprover takes a while (10s for [vampire](https://vprover.github.io/usage.html), a couple minutes for [eprover](https://github.com/eprover/eprover)) but it turns out running them without their command line arguments significantly slows them down. I think this has tripped me up before. These seems like very bad defaults. I used the included `ladr_to_tptp` tool to extract the following [tptp](https://www.tptp.org/) problem.

In [ ]:
%%file /tmp/sqrt.p

cnf(sos,axiom,m(one,A) = A).
cnf(sos,axiom,m(A,one) = A).
cnf(sos,axiom,m(A,m(B,C)) = m(m(A,B),C)).
cnf(sos,axiom,m(A,B) = m(B,A)).
cnf(sos,axiom,m(A,B) != m(A,C) | B = C).
cnf(sos,axiom,~ d(A,B) | m(A,f(A,B)) = B).
cnf(sos,axiom, m(A,B) != C | d(A,C)).
cnf(sos,axiom,~ d(two,m(A,B)) | d(two,A) | d(two,B)).
cnf(sos,axiom,m(a,a) = m(two,m(b,b))).
cnf(sos,axiom,~ d(A,a) | ~ d(A,b) | A = one).
cnf(sos,axiom,~ two = one).

Both of these succeed basically immediately, but the output is quite long.

In [1]:
! vampire --mode casc /tmp/sqrt.p

% Running in auto input_syntax mode. Trying TPTP
% lrs+10_11_cond=on:drc=off:flr=on:fsr=off:gsp=on:gs=on:gsem=off:lma=on:msp=off:nm=4:nwc=1.5:nicw=on:sas=z3:sims=off:sp=scramble:stl=188_1169 on sqrt for (600ds/0Mi)
perf_event_open failed (instruction limiting will be disabled): Permission denied
(If you are seeing 'Permission denied' ask your admin to run 'sudo sysctl -w kernel.perf_event_paranoid=-1' for you.)
% Refutation found. Thanks to Tanya!
% SZS status Unsatisfiable for sqrt
% SZS output start Proof for sqrt
fof(f39584,plain,(
  $false),
  inference(avatar_smt_refutation,[],[f17,f22,f49,f84,f118,f123,f132,f137,f143,f186,f192,f212,f262,f268,f318,f322,f326,f332,f353,f363,f384,f393,f398,f400,f411,f419,f461,f704,f761,f766,f772,f981,f1016,f1022,f1284,f1409,f1414,f1416,f1442,f1447,f1452,f1516,f1569,f1807,f2104,f3775,f3966,f3971,f3977,f4241,f4400,f4406,f4415,f4437,f4442,f4666,f4671,f4677,f4683,f4950,f4963,f5012,f5016,f5020,f5062,f5069,f5217,f5222,f5227,f5242,f5247,f5254,f5259,f5320,f5

In [2]:
! eprover-ho --auto-schedule --proof-object /tmp/sqrt.p

# Preprocessing class: FSSSSMSSSSSNFFN.
# Scheduled 1 strats onto 1 cores with 300 seconds (300 total)
# Starting G-E--_302_C18_F1_URBAN_RG_S04BN with 300s (1) cores
# G-E--_302_C18_F1_URBAN_RG_S04BN with pid 415821 completed with status 0
# Result found by G-E--_302_C18_F1_URBAN_RG_S04BN
# No SInE strategy applied
# Search class: FGUSM-FFSF22-SFFFFFNN
# Scheduled 6 strats onto 1 cores with 300 seconds (300 total)
# Starting SAT001_MinMin_p005000_rr_RG with 109s (1) cores
# SAT001_MinMin_p005000_rr_RG with pid 415822 completed with status 0
# Result found by SAT001_MinMin_p005000_rr_RG
# Initializing proof state
# Scanning for AC axioms
# m is AC
# AC handling enabled
#
#cnf(i_0_22, plain, (one!=two)).
#
#cnf(i_0_13, plain, (m(X1,one)=X1)).
#
#cnf(i_0_12, plain, (m(one,X1)=X1)).
#
#cnf(i_0_16, plain, (X2=X3|m(X1,X2)!=m(X1,X3))).
#
#cnf(i_0_21, plain, (X1=one|~d(X1,a)|~d(X1,b))).
#
#cnf(i_0_15, plain, (m(X1,X2)=m(X2,X1))).
#
#cnf(i_0_20, plain, (m(two,m(b,b))=m(a,a))).
#
#cnf(i_0_18, pl

Anyhow, with that guidance, we can attempt to write a proof like thing in python.

I've been tinkering with trying to enforce a proof discipline / kernel in python, but I've been leaning towards reducing it all to idioms until I'm more sure what I want. 

- https://github.com/philzook58/knuckledragger github
- https://www.philipzucker.com/knuckledrag2/ second blog post
- https://www.philipzucker.com/python-itp/ first blog post

The kernel is only enforcing rules that I think make sense anyway.

A small wrapper enables us the make stepwise proofs. We only give python names to things that we want to accept as axioms or things that come out of `lemma`. The justifications that go into `by` should only be previously named theorems.

This avoids all the rigamorole of keeping a separate `Theorem` datatype or theorem database, but is more susceptible to accidental error than the styles I've used in those other posts.

In [12]:
def lemma(thm, by=[]):
    prove(Implies(And(by), thm))
    return thm

The following let's me see last assignment, prewtty printing the statement of the lemma in a nice form.

In [14]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"

The strategy is to make opaque functions that we can hide the definitions that may be confusing the solver. The approach semi fails.

In [23]:
Z = IntSort()
B = BoolSort()
x,y,z = Ints("x y z")
mul = Function("mul", Z, Z, Z)
mul_def = ForAll([x,y], mul(x, y) == x*y)

proved


ForAll(x, mul(x, 1) == x)

Z3 easily proves the basic algebraic properties of multiplication

In [33]:
mul_x_1 = lemma(ForAll([x], mul(x, 1) == x), by=[mul_def])

proved


ForAll(x, mul(x, 1) == x)

In [17]:
mul_1_x = lemma(ForAll([x], mul(1, x) == x), by=[mul_def])

proved


ForAll(x, mul(1, x) == x)

In [18]:
mul_assoc = lemma(ForAll([x,y,z], mul(x, mul(y,z)) == mul(mul(x,y), z)), by=[mul_def])

proved


ForAll([x, y, z], mul(x, mul(y, z)) == mul(mul(x, y), z))

In [19]:
mul_comm = lemma(ForAll([x,y], mul(x,y) == mul(y,x)), by=[mul_def])

proved


ForAll([x, y], mul(x, y) == mul(y, x))

In [20]:
mul_cancel = lemma(ForAll([x,y,z], Implies(And(x != 0, mul(x,y) == mul(x,z)), y == z)), by=[mul_def])

proved


ForAll([x, y, z],
       Implies(And(x != 0, mul(x, y) == mul(x, z)), y == z))

We can define a [divides](https://en.wikipedia.org/wiki/Divisor) relation. I define it in terms of `x/y` which is flooring division. Is there a built in divides relation in SMT?

In [24]:
divides = Function("divides", Z, Z, B)
divides_def = ForAll([x,y], divides(x,y) == (x == y*(x/y)))

ForAll([x, y], divides(x, y) == (x == y*(x/y)))

In [26]:
div_mul_cancel = lemma(ForAll([x,y], Implies(divides(x,y), mul(x/y, y) == x)), by=[divides_def, mul_def])

proved


ForAll([x, y], Implies(divides(x, y), mul(x/y, y) == x))

Here we have two properties that I can't get to pass. They are basic facts. We don't get a counterexample, so perhaps that lends a mild amount of credence.

In [29]:
#mul_divides = lemma(ForAll([x,y,z], Implies(mul(x,y) == z, divides(z,x))), by=[mul_def, divides_def])
# can't get this one to prove
mul_divides = ForAll([x,y,z], Implies(mul(x,y) == z, divides(z,x)))

ForAll([x, y, z], Implies(mul(x, y) == z, divides(z, x)))

In [27]:
#two_prime = lemma(ForAll([x,y], Implies(divides(mul(x,y),2), Or(divides(x,2), divides(y,2)))), by=[divides_def, mul_def])
# can't get this one to pass
two_prime = ForAll([x,y], Implies(divides(mul(x,y),2), Or(divides(x,2), divides(y,2))))

ForAll([x, y],
       Implies(divides(mul(x, y), 2),
               Or(divides(x, 2), divides(y, 2))))

Part of the cleverness of the Otter proof is abusing this notion that every fraction can be expressed in lowest common terms. This is a nontrivial seeming fact.

Here I use an expanded form of `solve` because I was fiddling with the smt formula.

In [28]:
no_common_factor = ForAll([z], Implies(And(divides(x,z), divides(y,z)), z == 1))
prime_thm = Not(Exists([x,y], And(no_common_factor, x != 0, mul(x,x) == mul(2,mul(y,y)))))
lemmas = [mul_1_x, mul_x_1, mul_comm, mul_assoc, mul_divides, mul_cancel, div_mul_cancel, two_prime]
s = Solver()
s.add(lemmas)
s.add(Not(prime_thm))
with open("/tmp/prime.smt2", "w") as f:
    f.write(s.sexpr()) 
#lemma(Not(Exists([x,y], And(no_factor, mul(x,x) == mul(2,mul(y,y))))), by=[mul_1_x, mul_x_1, mul_assoc, two_prime, div_mul_cancel])
s.check()

unsat

## Bits and Bobbles

I am not satisfied. There are big holes and there was some nontrivial workarounds. I'm not particularly sure my thrashing even helped that much.



In [31]:
!vampire --mode tclausify /tmp/prime.smt2  # vampire is useful for converting to clauses or smt to tptp

% Running in auto input_syntax mode. Trying SMTLIB2
tff(func_def_0, type, mul: ($int * $int) > $int).
tff(func_def_12, type, sK3: $int).
tff(func_def_13, type, sK4: $int).
tff(pred_def_1, type, divides: ($int * $int) > $o).
tff(pred_def_5, type, lG2: ($int * $int) > $o).
tff(u63,axiom,
    (lG2(sK4,sK3) | ~sLP1)).

tff(u62,axiom,
    ((0 != sK3) | ~sLP1)).

tff(u61,axiom,
    ((mul(2,mul(sK4,sK4)) = mul(sK3,sK3)) | ~sLP1)).

tff(u64,axiom,
    (![X2 : $int, X0 : $int, X1 : $int] : (((1 = X2) | ~divides(X1,X2) | ~divides(X0,X2) | ~lG2(X0,X1))))).

tff(u35,axiom,
    (![X0 : $o] : (((($true = X0)) | (($false = X0)))))).

tff(u34,axiom,
    (($true != $false))).

tff(u33,axiom,
    (![X0 : $int, X1 : $int] : ((~$less(X0,X1) | ~$less(X1,$sum(X0,1)))))).

tff(u32,axiom,
    (![X2 : $int, X1 : $int] : (((0 = X2) | ~$less($sum($abs(X2),$uminus(1)),$remainder_e(X1,X2)))))).

tff(u31,axiom,
    (![X2 : $int, X1 : $int] : (((0 = X2) | ~$less($remainder_e(X1,X2),0))))).

tff(u30,axiom,
    (![X2 

I think I have less interest in rigor than others who might be interested in such things.


I was once stumped on how to derive the irrationality of the square root of 2 in Z3. Here I'm going to recpliate it.
I suspect ATP systems like vampire or eprover will not get stuck

It can be formulated as proving `not exists m n, n^2 = 2 m^2`  $\neg \exists m n, n^2 = 2 m^2$

Cody also suggested infinitude of primes.

This is all math shit. These are not my theorems. I think maybe they are the wrong ones to attack.

Facts about evens and odds.

In [1]:
#from cvc5.pythonic import *
from z3 import *
x,y,z = Ints("x y z")
p = Real("p")
Z = IntSort()
R = RealSort()
B = BoolSort()

The paper mentions that many systems don't have the reals. Z3 and CVC5 do. So you can state something closer to what you meant.

In a certain sense, the SMT "reals" are not the reals at all. They can prove some under approximation of reasonable facts about the reals. They are based around the rationals, linear equations, linear inequalities, polynomials and algebraic numbers.


In [2]:
rational = Function("rational", R, B)
rat_def = ForAll([p], rational(p) == Exists([x,y], p == ToReal(x)/ToReal(y)))
rat_def

ForAll(p,
       (Exists([x, y], p == ToReal(x)/ToReal(y))) ==
       rational(p))

In [3]:
step1 = Implies(And(rational(p), p*p == 2), Exists([x,y], x*x == 2*y*y))
prove(Implies(rat_def, step1))


In [ ]:
#prove(ForAll([p], Implies(rational(p), Not(p*p == 2))))

NameError: name 'rational' is not defined

Lemmas about even and oddness

I have a suspicion that it is wise to define things in "open" and explicitly skolemized form.

`evenp(x,y)` is the "evidence" form of evenness, where you give explicitly the number that is half of x. This perhaps gives you the option of helping the solver find the needed existential by just giving it.
 `even(x) = exists p, evenp(x,p)`

Explicit skolem function help you refer to something that is guaranteed to exist.

Choice functions would make this easier probably.



In [ ]:
evenp = Function("evenp", I, I, B)
evenp_def = ForAll([x,y], evenp(x,y) == (x == 2*y))


xsq_even = Implies(x*x == 2*y*y, evenp(x*x, y*y))
prove(Implies(evenp_def, xsq_even))


ForAll([x,y], Implies( evenp(x,y), even(x)))
#ForAll([x,y], Implies( evenp(x,y), even(y)))







In [3]:
rearrange = Implies(x*x == 2*y*y, y*y == (x*x)/2)
prove(rearrange)

proved


In [ ]:
rearrange = Implies(x*x == 2*y*y, y*y == (x*x)/2)
prove(rearrange)

In [ ]:
xsq_even = Implies(x*x == 2*y, even(x*x))
prove(Implies(even_def, xsq_even))
xsq_even

In [ ]:
#from cvc5.pythonic import *
#from z3 
x,y,z = Ints("x y z")
even = Function("even", IntSort(), BoolSort())
even_def = ForAll(x, even(x) == Exists([y], x == 2*y))
odd = Function("odd", IntSort(), BoolSort())
odd_def = ForAll(x, odd(x) == Exists([y], x == 2*y+1))

even_not_odd = ForAll([x], Implies(even(x), Not(odd(x))))
prove(Implies(And(even_def, odd_def), even_not_odd))

even_or_odd = ForAll(x, even(x) != odd(x))
prove(Implies(And(even_def, odd_def), even_or_odd))



proved
proved
proved


Facts about even odd

In [ ]:
even_not_odd = ForAll(x, Implies(even(x), Not(odd(x))))
prove(Implies(And(even_def, odd_def), even_not_odd))

odd_not_even = ForAll(x, Implies(odd(x), Not(even(x))))
prove(Implies(And(even_def, odd_def), odd_not_even))


not_even_odd = ForAll(x, Not(And(even(x),odd(x))))
prove(Implies(And(even_def, odd_def), not_even_odd))




def induct(P):
    return Implies(
            And(P(0), 
                ForAll([x], Implies(P(x),  And(P(x + 1), P(x - 1))))),
            ForAll([x], P(x)))

even_or_odd = ForAll(x, Or(even(x), odd(x)))
#prove(Implies(And(even_def, odd_def, induct(lambda n: Or(even(n), odd(n)))), even_or_odd))

evenodd = lambda n: Or(even(n), odd(n))
prove(Implies(And(even_def, odd_def), Or(even(0), odd(0))))
prove(Implies(And(even_def, odd_def), ForAll([x], Implies(evenodd(x), evenodd(x+1)))))
#prove(Implies(And(even_def, odd_def), ForAll([x], Implies(evenodd(x), evenodd(x-1)))))

prove(Implies(And(even_def, odd_def), ForAll([x], Implies(odd(x), Exists([y], x == 2*y-1)))))

#prove(Implies(And(even_def, odd_def), ForAll([x], odd(x) == Exists([y], x == 2*y-1))))

even_or_odd = ForAll(x, even(x) != odd(x))
# Z3 can't prove this for some reason. I suppose this might be a complex diophantine equation or something.
prove(Implies(And(even_def, odd_def), even_or_odd))


sqrt_thm = ForAll([x,y], Implies(x != 0, 2*x*x != y*y))
#prove(sqrt_thm)

#solve(2*x*x == y*y)

even_sq = ForAll([x,y], Implies(even(x), even(x*x)))
#prove(Implies(even_def, even_sq))




In [3]:
#def divides(x,y):
#    return x == y*(x/y)

divides = Function("divides", I, I, B)
divides_def = ForAll([x,y], divides(x,y) == (x == y*(x/y)))
prove(ForAll([x], divides(x, 0)))



NameError: name 'Function' is not defined

In [89]:
%%file /tmp/prime2.smt2
(declare-sort I 0)
(declare-fun mul (I I) I)
(declare-fun divsk (I I) I)
(declare-fun divides (I I) Bool)
(declare-const one I)
(declare-const two I)
(declare-const a I)
(declare-const b I)
(assert (distinct one two))
(assert (forall ((x I)) (= (mul one x) x)))
(assert (forall ((x I)) (= (mul x one) x)))
(assert (forall ((x I) (y I) (z I)) (= (mul x (mul y z)) (mul (mul x y) z))))
(assert (forall ((x I) (y I)) (= (mul x y) (mul y x))))
(assert (forall ((x I) (y I) (z I))
  (=> (= (mul x y) (mul x z)) (= y z))))
(assert (forall ((x I) (y I)) (=> (divides x y) (= (mul y (divsk x y)) x))))
(assert (forall ((x I) (y I))
  (=> (divides (mul x y) two) (or (divides x two) (divides y two)))))
(assert (= (mul a a) (mul two (mul b b)))) ; 
(assert (forall ((x I)) (=> (and (divides a x) (divides b x)) (= x one)))) ; lowest common terms
;(assert (let ((a!1 (exists ((x I) (y I))
;             (let ((a!1 (forall ((z I))
;                         (=> (and (divides x z) (divides y z)) (= z one)))))
;               (and a!1 (= (mul x x) (mul two (mul y y))))))))
;  (not (not a!1))))

Writing /tmp/prime2.smt2


Overwriting /tmp/sqrt2.in


In [66]:
! prover9 -f /tmp/sqrt2.in

============================== Prover9 ===============================
Prover9 (64) version 2017-11A (CIIRC), November 2017.
Process 259687 was started by philip on philip-ThinkPad-Z13-Gen-2,
Mon Apr 29 17:27:10 2024
The command was "prover9 -f /tmp/sqrt2.in".
============================== end of head ===========================

============================== INPUT =================================

% Reading from file /tmp/sqrt2.in

set(auto).
    % set(auto) -> set(auto_inference).
    % set(auto) -> set(auto_setup).
    % set(auto_setup) -> set(predicate_elim).
    % set(auto_setup) -> assign(eq_defs, unfold).
    % set(auto) -> set(auto_limits).
    % set(auto_limits) -> assign(max_weight, "100.000").
    % set(auto_limits) -> assign(sos_limit, 20000).
    % set(auto) -> set(auto_denials).
    % set(auto) -> set(auto_process).

formulas(sos).
x = x.
m(1,x) = x.
m(x,1) = x.
m(x,m(y,z)) = m(m(x,y),z).
m(x,y) = m(y,x).
m(x,y) != m(x,z) | y = z.
-d(x,y) | m(x,f(x,y)) = y.
m(x,z) != y

Overwriting /tmp/sqrt.p


In [94]:
%%file /tmp/prime3.smt2
(declare-sort I 0)
(declare-fun m (I I) I)
(declare-fun f (I I) I)
(declare-fun d (I I) Bool)
(declare-const one I)
(declare-const two I)
(declare-const a I)
(declare-const b I)

(assert (forall ((x I)) (= (m one x) x)))
(assert (forall ((x I)) (= (m x one) x)))
(assert (forall ((x I) (y I) (z I)) (= (m x (m y z)) (m (m x y) z))))
(assert (forall ((x I) (y I)) (= (m x y) (m y x))))
(assert (forall ((x I) (y I) (z I))
  (=> (= (m x y) (m x z)) (= y z))))
(assert (forall ((x I) (y I)) (=> (d y x) (= (m y (f y x)) x))))
(assert (forall ((x I) (y I))
  (=> (d two (m x y)) (or (d two x) (d two y)))))
(assert (= (m a a) (m two (m b b)))) ; 
(assert (forall ((x I)) (=> (and (d x a) (d x b)) (= x one)))) ; lowest common terms
(assert (distinct one two))
(check-sat)

Overwriting /tmp/prime3.smt2


In [88]:
! time vampire /tmp/sqrt.p

% Running in auto input_syntax mode. Trying TPTP
% Refutation found. Thanks to Tanya!
% SZS status Unsatisfiable for sqrt
% SZS output start Proof for sqrt
3. m(X0,m(X1,X2)) = m(m(X0,X1),X2) [input]
4. m(X0,X1) = m(X1,X0) [input]
5. m(X0,X1) != m(X0,X2) | X1 = X2 [input]
6. ~d(X0,X1) | m(X0,f(X0,X1)) = X1 [input]
7. d(X0,X2) | m(X0,X1) != X2 [input]
8. ~d(two,m(X0,X1)) | d(two,X0) | d(two,X1) [input]
9. m(a,a) = m(two,m(b,b)) [input]
10. ~d(X0,b) | one = X0 | ~d(X0,a) [input]
11. one != two [input]
12. d(X0,m(X0,X1)) [equality resolution 7]
21. d(X0,m(X1,X0)) [superposition 12,4]
25. d(two,m(a,a)) [superposition 12,9]
46. m(a,a) != m(two,X0) | m(b,b) = X0 [superposition 5,9]
53. m(X0,X1) = m(X0,f(X0,m(X0,X1))) [resolution 6,12]
57. m(a,a) = m(two,f(two,m(a,a))) [resolution 6,25]
74. d(X2,m(X0,m(X1,X2))) [superposition 21,3]
88. d(two,a) | d(two,a) [resolution 8,25]
98. d(two,a) [duplicate literal removal 88]
99. a = m(two,f(two,a)) [resolution 98,6]
198. m(X0,X1) != m(X0,X2) | f(X0,m(X

In [107]:
%%file /tmp/sqrtcluase.p

cnf(sos,axiom,
    m(one,X0) = X0).

cnf(sos,axiom,
    m(X0,one) = X0).

cnf(sos,axiom,
    m(m(X1,X0),X2) = m(X1,m(X0,X2))).

cnf(sos,axiom,
    m(X0,X1) = m(X1,X0)).

cnf(sos,axiom,
    X0 = X2 | m(X1,X2) != m(X1,X0)).

cnf(sos,axiom,
    m(X0,f(X0,X1)) = X1 | ~d(X0,X1)).

cnf(sos,axiom,
    d(two,X1) | d(two,X0) | ~d(two,m(X1,X0))).

cnf(sos,axiom,
    m(two,m(b,b)) = m(a,a)).

cnf(sos,axiom,
    one = X0 | ~d(X0,a) | ~d(X0,b)).

cnf(sos,axiom,
    one != two).

%cnf(sos,axiom,~ d(A,B) | m(A,f(A,B)) = B).
cnf(sos,axiom, m(A,B) != C | d(A,C)).
%cnf(sos,axiom,~ d(two,m(A,B)) | d(two,A) | d(two,B)).
%cnf(sos,axiom,m(a,a) = m(two,m(b,b))).
%cnf(sos,axiom,~ d(A,a) | ~ d(A,b) | A = one).
%cnf(sos,axiom,~ two = one).

Overwriting /tmp/sqrtcluase.p


In [ ]:
! time eprover-ho /tmp/sqrt.p

In [21]:
import z3

x = z3.Int("x")
e = x+2
e.sexpr()
x.decl().sexpr()
def collect_symbols(expr, symbols):
    """ Recursively collect constants and function symbols from an expression. """
    if expr.num_args() == 0:
        # It's a constant if it has no arguments (excluding bound variables in quantifiers)
        if expr.decl().kind() == z3.Z3_OP_UNINTERPRETED:
            symbols.add(expr)
    else:
        # If the node is a function application, add the function declaration
        if expr.decl().kind() == z3.Z3_OP_UNINTERPRETED or expr.decl().kind() == z3.Z3_OP_APPLY:
            symbols.add(expr.decl())
        # Go deeper into the expression for nested function applications
        for i in range(expr.num_args()):
            collect_symbols(expr.arg(i), symbols)
# naw this is ridiculous. I'll just collect up my constants manually.
#collect_symbols(e, set())
z3.ForAll([x],e > 2).sexpr()
Nat = z3.Datatype("Nat")
Nat.declare("zero")
Nat.declare("succ", ("prev", Nat))
Nat = Nat.create()
Nat.sexpr()
Nat.succ.sexpr()

'(declare-fun succ (Nat) Nat)'

## PySMT
I thought maybe I should use pysmt. But I dunno. I'm so familiar with the z3 bindings.


In [3]:
from pysmt.shortcuts import Symbol, And, GE, LT, Plus, Equals, Int, get_model
from pysmt.typing import INT

hello = [Symbol(s, INT) for s in "hello"]
world = [Symbol(s, INT) for s in "world"]
letters = set(hello+world)
domains = And([And(GE(l, Int(1)),
                   LT(l, Int(10))) for l in letters])

sum_hello = Plus(hello) # n-ary operators can take lists
sum_world = Plus(world) # as arguments
problem = And(Equals(sum_hello, sum_world),
              Equals(sum_hello, Int(25)))
formula = And(domains, problem)

print("Serialization of the formula:")
print(formula)

model = get_model(formula)
if model:
  print(model)
else:
  print("No solution found")

Serialization of the formula:
((((1 <= h) & (h < 10)) & ((1 <= e) & (e < 10)) & ((1 <= l) & (l < 10)) & ((1 <= o) & (o < 10)) & ((1 <= w) & (w < 10)) & ((1 <= r) & (r < 10)) & ((1 <= d) & (d < 10))) & (((h + e + l + l + o) = (w + o + r + l + d)) & ((h + e + l + l + o) = 25)))
d := 9
o := 5
r := 1
w := 1
h := 1
l := 9
e := 1


In [1]:
from pysmt.logics import QF_UFLRA, QF_UFIDL, QF_LRA, QF_IDL, QF_LIA
from pysmt.shortcuts import get_env, GT, Solver, Symbol
logics = [QF_UFLRA, QF_UFIDL, QF_LIA],  # Some of the supported logics

env = get_env()

# Add the solver to the environment
env.factory.add_generic_solver("vampire","/usr/local/bin/vampire" , logics)

In [2]:
with Solver(name="vampire", logic=QF_LIA) as s:
    s.add_assertion(formula)
    if s.solve():
        print(s.get_model())
    else:
        print("No solution found")

AttributeError: 'list' object has no attribute 'theory'


### A Manual Proof Discipline
Here were some old experiments using raw smtlib idioms

Separation between module signature and proofs. Seperate files.
Proof datatype to separate from bool definitions

```z3
;re
(declare-datatypes () (
  (Proof (Proof (getformula Bool)))
))
(define-fun proved ((p Proof)) Bool 
  (match p (
    ((Proof p) (not p))
  ))
)
(define-fun instan-lemma1 ((x Int)) Bool  (or (> x 0) (< x 0) (= x 0)))
(define-const lemma1 Proof (Proof (forall ((x Int)) (instan-lemma1 x))))

(push)
;------------
(assert (proved lemma1))
(check-sat)
(pop)

```

Other possibility for Proof type
Forall (Expr)
Exists (Expr)
Eh. I don't really want to reflect everything.

A shallower HOAS embedding

```z3
;re
(declare-datatype Formula (
  (Bool (bool1 Bool))
  (Forall (forall1 (Array Int Formula)))
  (Exists (exists1 (Array Int Formula)))
))


```

```z3
;re
(declare-datatype Exists 
  (
    (Pair (x Int) (pred (Array Int Bool)))
  ))

```

Universals can be proven by having
(define-const x)

```
; other hypothesis do not mention x
;-----------------
(instan-lemma1 x)
```

or by
; other hypothesis can't possibly mention x
;-----------
(assert (proved lemma1))

Universal can be used by using lemma1 or by instan-lemma1 on anything

Existentials can be proved

```
----------
(exists x (relx x))
```

or by

```
whatever
---------
(relx 3)

```

They can be used via

```
define-const x
(relx x)
; no other use of x
;----------------
conclusion may refer to x

```

ExistsE (ExistsE Bool)
ForallE (ForallE (forallE Bool)) ; (apply?)
Forall (ForallI (forallE Bool))

(Exists Int Bool) ; can I pack the witness?

define-const lemma Forall (forall x (forallE (instan-lemma x)))
define-fun instan-lemma ((x Int)) ForallE

The method is related to lambda lifting.
We need to give names to different free parameter things.

```z3
;re
(declare-fun even-fun (Int) Int)
(define-fun even ((x Int)) Bool (= (* 2 (even-fun x)) x))

(declare-fun odd-fun (Int) Int)
(define-fun odd ((x Int)) Bool (= (+ 1 (* 2 (odd-fun x))) x))

;(define-fun even ((x Int)) Bool (exists ((y Int)) (= (* 2 y) x)))
(define-const even-axiom Bool (forall ((x Int)) (=> (exists ((y Int)) (= (* 2 y) x)) (= (* 2 (even-fun x)) x))))
;(define-fun even ((x Int)) Bool (exists ((y Int)) (and (= y (even-fun x)) (= (* 2 y) x))))
(define-fun not-even ((x Int)) Bool (forall ((y Int)) (distinct x (* 2 y))))
;(define-fun instan-not-even ((x Int) (y Int)) (distinct x (2 * y)))

(declare-const m Int)
(declare-const n Int)
(assert (odd m))
(assert (odd n))
(assert (not (even m)))
(assert (not (even n)))
;(assert (instan-not-even m (even-fun )))
(assert-not (even (+ n m)))
;(assert (not-even (+ n m)))
(check-sat)
(get-model)

```

```z3
;re
(define-fun even ((x Int) (y Int)) Bool (= (* 2 y) x))
(define-fun odd ((x Int) (y Int)) Bool (= (+ 1 (* 2 y)) x))


(declare-const m Int)
(declare-const n Int)
(declare-const mo Int)
(declare-const no Int)
(assert (odd m mo))
(assert (odd n no))
(assert-not (exists ((x Int)) (even (+ m n) x)))
(check-sat)

```

```z3
;re
(define-fun even ((x Int)) Bool (exists ((y Int)) (= (* 2 y) x)))
(define-fun odd ((x Int)) Bool (exists ((y Int)) (= (+ 1 (* 2 y)) x)))

(declare-const m Int)
(declare-const n Int)
(assert (odd m))
(assert (odd n))
;--------------------------
(assert-not (even (+ m n)))
(check-sat)
```

```z3

```z3
;re
(define-fun even ((x Int)) Bool (exists ((y Int)) (= (* 2 y) x)))
(define-fun div4 ((x Int)) Bool (exists ((y Int)) (= (* 4 y) x)))

(declare-const m Int)
(assert (not (even m)))
;---------------------------
(assert-not (not (div4 m)))
(check-sat)
```

Positively asserting existence isn't so bad. It is just asserting on a fresh variable.

Only allow mention variable once on existential elim in hyps

I'm a little worried about circulaity if I pack everything into one proof thing.

existential lemma can be used in hyps.
Instantiated version in a conc (give explicitly). Or just let it crank.

```
(declare-const e) appears only once in hyp for existential elim

(assert-not
  (=> (and (existse e) (instan-lemma1 ))
      (and lemma1 )
  )

)

```

```z3
;re
(define-fun instan-theorem ((m Int) (n Int)) Bool (not (= (* m m) (* 2 (* n n)))))
(define-const theorem Bool (forall ((m Int) (n Int)) (not (= (* m m) (* 2 (* n n))))))


```

### Python Hilbert Proof
An older database style attempt

Hilbert style proofs where the only inference rule is Z3_check.
Register known theorems to central repository. Refer to them later.
There are some basic theorems with quantifiers I couldn't get Z3 to do even the most basic step of, so this idea feels sunk.
I guess I could also admit vampire.

```python
from z3 import *
class Kernel():
    def __init__(self):
        self._formula = {}
        self._schema = {}
        self._explanations = {}
    def axiom(self, name, formula):
        assert name not in self._formula
        self._formula[name] = (formula, None)
        return name
    def add_schema(self, name, schema):
        assert name not in self._schema
        self._schema[name] = schema
        return name
    def instan_schema(self, schema, name, *args):
        #name = self.fresh(name)
        self._formula[name] = (self._schema[schema](*args), ("schema", schema))
        return name
    def theorem(self, name, formula, *reasons, timeout=1):
        assert name not in self._formula
        s = Solver()
        #s.set("timeout", timeout)

        s.add([ self._formula[reason][0] for reason in reasons])
        # snity check here. s.check(). should return unknown. If returns unsat, we have an incosisitency
        # Even if we have inconsistency but z3 isn't finding it, that is a small comfort. 
        s.add(Not(formula))
        status = s.check()
        if status == unsat:
            self._formula[name] = (formula, reasons)
            print(f"Accepted {name}")
        elif status == sat:
            print(f"Counterexample : {s.model()}")
        elif status == unknown:
            print("Failed: reason unknown")
        else:
            print(f"unexpected status {status}")
    #def calc(self):
    #    pass
    #def definition: # is definition distinct from axiom?

t = Kernel()
p = Real("p")
t.axiom("p_def", p*p == 2)
m,n = [ToReal(x) for x in Ints("m n")]

#t.theorem("p irrational", p != m / n)
#(m / n == p)
def even(x):
    y = FreshInt()
    return Exists([y], 2 * y == x)
def odd(x):
    y = FreshInt()
    return Exists([y], 2 * y + 1 == x)
r,w = Ints("r w")
t.theorem("even odd", ForAll([r], even(r) != odd(r)))
def rational(p):
    m = FreshInt("m")
    n = FreshInt("n")
    return Exists([m,n], And(n != 0, p == ToReal(m) / ToReal(n) ))
rational(p)

#t.theorem("mndef", Implies(rational(p), p == m / n) )

#t.theorem( Implies( p == m / n, Exists([r,w], And(p == r / w,  Or(And(even(r), odd(w)), And(odd(r), even(w)) ) ))))
# how do you use this exists?

#t.theorem(   )
def evenodd(n,m):
    return even(n) != even(m)

t.theorem(0, Implies(  And(p == m / n, n != 0) , p * n == m  ))
t.theorem(1, Implies(  And(p == m / n, n != 0, evenodd(m,n)) , p * n == m  ))
#t.theorem(2, Implies(  And(p == m / n, n != 0, evenodd(m,n)) , 2 * n * n == m * m , "p_def"))
t.theorem(3, even(r) != odd(r))
#t.theorem(4, even(r * r) != odd(r * r))
t.theorem(5, Implies(r == 2 * w, even(r*r) ))
t.theorem(6, Implies(r == 2 * w, r*r == 4 * w * w ))

#t.theorem(7, Implies(r == 2 * w, even(r) ))
#t.theorem(8, Implies(even(r), even(r*r)), 5, 6, 7)
t.theorem("even or odd", Or(even(r), odd(r)))
#t.theorem("even or odd r*r", Or(even(r*r), odd(r*r)))
t.theorem(8, ForAll([r], even(r) != odd(r)))
t.theorem(9, even(r*r) != odd(r*r), 8)
# I dunno. This idea is sunk. I can't even get this to work?
# I could try not using the built in stuff I guess and wwork in FOL.


#t.theorem(5, odd(r) == odd(r * r), 3, 4)
#t.theorem(3, Implies(even(r * r), even(r) ))
#t.theorem(3, Implies(  And(2 * n * n == m * m), even(m)))




#t.theorem(0, Implies(  p == m / n , m * m == 2 * n * n   ), "p_def" )
even(r * r) != odd(r * r)
```

```python
t = Hilbert()

Nat = Datatype('Nat')
Nat.declare('Zero')
Nat.declare('Succ', ('pred', Nat))
# Create the datatype
Nat = Nat.create()
Zero = Nat.Zero
Succ = Nat.Succ

n,m = Consts("n m", Nat)
plus = Function("+", Nat,Nat,Nat)
DatatypeRef.__add__ = lambda self, rhs : plus(self, rhs)
#type(n)
plus_zero = t.axiom("plus_zero", ForAll([m], Zero + m == m))
plus_succ = t.axiom("plus_succ", ForAll([n,m], Succ(n) + m == Succ(n + m)))
plus_def = [plus_zero, plus_succ]
def induction_schema(P):
    return  Implies(
                And(P(Zero), 
                    ForAll([n], Implies(P(n), P(Succ(n))))) ,
                ForAll([n], P(n))
    )
induction = t.add_schema("induction", induction_schema)

t.theorem( "plus_zero'",  ForAll([m], Zero + m == m , patterns=[Zero + m]), plus_zero) # guardedc version of the axiom
mylemma = t.instan(induction, "mylemma", lambda m : Zero + m == m + Zero )

t.theorem( "n_plus_zero", ForAll([m], Zero + m == m + Zero) ,  plus_zero, plus_succ,  mylemma )
t.theorem("test guarding", Zero + Zero == Succ(Zero), "plus_zero")

lte = Function("<=", Nat, Nat, BoolSort())
DatatypeRef.__le__ = lambda self, rhs : lte(self, rhs)

t.axiom("zero lte", ForAll([n], Zero <= n))
t.axiom("succ lte", ForAll([n,m], (Succ(n) <= Succ(m)) == (n <= m) ))
lte_def = ["zero lte", "succ lte"]


t.theorem("example",  (Zero + Zero) <= Zero,  *plus_def, *lte_def  )


reflect = Function("reflect", Nat  , IntSort())
x = Int("x")
t.axiom( "reflect succ", ForAll([x, n], (reflect(Succ(n)) == 1 + x) == (reflect(n) == x) ))
t.axiom( "reflect zero", reflect(Zero) == 0) 

t.formula

```